# Chương 3: Các mô hình Dự báo và Phân loại (Kỹ thuật Phụ thuộc)

**Mục tiêu:** Trang bị các công cụ để dự báo một biến mục tiêu (Dependent Variable - $Y$) dựa trên các biến giải thích (Independent Variables - $X$).

Chúng ta chia làm 2 nhánh chính:
1.  **Dự báo định lượng (Regression):** Khi $Y$ là con số liên tục (Ví dụ: Doanh thu, Lợi nhuận, Giá nhà).
2.  **Dự báo phân loại (Classification):** Khi $Y$ là nhãn nhóm (Ví dụ: Vỡ nợ/Không vỡ nợ, Mua/Không mua).

---

## 3.1. Hồi quy Tuyến tính Bội (Multiple Linear Regression)

### Case Study 1: Định giá bất động sản
Một công ty bất động sản muốn xây dựng công thức định giá nhà tự động dựa trên các đặc điểm: Diện tích ($X_1$), Số phòng ngủ ($X_2$), và Khoảng cách đến trung tâm ($X_3$).

### 3.1.1. Mô hình Toán học
Mô hình tổng quát:
$$ Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p + \epsilon $$

Trong đó:
* $\beta_0$: Hệ số chặn (Intercept).
* $\beta_i$: Hệ số hồi quy riêng phần. Nó cho biết $Y$ thay đổi bao nhiêu đơn vị khi $X_i$ tăng 1 đơn vị (giữ các biến khác không đổi).
* $\epsilon$: Sai số ngẫu nhiên (Residuals).

### 3.1.2. Đánh giá mô hình
Chúng ta quan tâm đến:
1.  **$R^2$ (R-squared):** Mô hình giải thích được bao nhiêu % sự biến thiên của $Y$?
2.  **P-value của từng biến:** Biến $X_i$ có thực sự ảnh hưởng đến $Y$ hay không (hay chỉ do ngẫu nhiên)?
3.  **F-test:** Toàn bộ mô hình có ý nghĩa thống kê không?

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# --- GIẢ LẬP DỮ LIỆU GIÁ NHÀ ---
np.random.seed(10)
n_samples = 200

# Biến độc lập X
area = np.random.normal(100, 20, n_samples)       # Diện tích (m2)
rooms = np.random.randint(1, 6, n_samples)        # Số phòng ngủ
dist_center = np.random.uniform(1, 20, n_samples) # Khoảng cách đến trung tâm (km)

# Biến phụ thuộc Y (Giá nhà - tỷ VND)
# Công thức thực tế (Ground Truth): Giá = 2 + 0.05*Area + 0.5*Rooms - 0.2*Distance + Noise
price = 2 + 0.05 * area + 0.5 * rooms - 0.2 * dist_center + np.random.normal(0, 0.5, n_samples)

df_house = pd.DataFrame({'Price': price, 'Area': area, 'Rooms': rooms, 'Distance': dist_center})

print("Dữ liệu mẫu giá nhà:")
print(df_house.head())

In [ ]:
# --- CHẠY HỒI QUY BẰNG STATSMODELS ---
# Statsmodels cho bảng kết quả thống kê chi tiết như SPSS/Stata

# Thêm hằng số chặn (Intercept) vào X
X = df_house[['Area', 'Rooms', 'Distance']]
X_with_const = sm.add_constant(X)
y = df_house['Price']

# Khởi tạo và huấn luyện mô hình OLS (Ordinary Least Squares)
model_ols = sm.OLS(y, X_with_const).fit()

# In bảng kết quả
print(model_ols.summary())

### 3.1.3. Diễn giải kết quả (Interpretation)
*Sinh viên nhìn vào bảng Summary ở trên và đọc các chỉ số:*

1.  **R-squared:** Giả sử ra 0.85 -> Các biến diện tích, số phòng, vị trí giải thích được 85% sự biến động của giá nhà.
2.  **Coef (Hệ số):**
    * `Distance`: Hệ số âm (ví dụ -0.19) -> Nhà càng xa trung tâm, giá càng giảm.
    * `Area`: Hệ số dương -> Nhà càng rộng càng đắt.
3.  **P>|t| (P-value):** Nếu giá trị này < 0.05, biến đó có ý nghĩa thống kê.

---

## 3.2. Hồi quy Logistic (Logistic Regression)

### Case Study 2: Chấm điểm tín dụng (Credit Scoring)
Ngân hàng cần quyết định: Có nên cho khách hàng A vay vốn không? (Có/Không). Biến phụ thuộc $Y$ lúc này chỉ nhận giá trị 0 hoặc 1.

### 3.2.1. Tại sao không dùng Hồi quy tuyến tính?
Nếu dùng hồi quy tuyến tính, kết quả $Y$ có thể ra 1.5 hoặc -0.2 (vô nghĩa về mặt xác suất). Hồi quy Logistic sử dụng hàm **Sigmoid** để nén kết quả về khoảng $(0, 1)$.

Công thức xác suất:
$$ P(Y=1) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 X_1 + \dots)}} $$

### 3.2.2. Odds Ratio (Tỷ số chênh)
Trong Logistic, hệ số $\beta$ hơi khó đọc. Chúng ta thường dùng $e^{\beta}$ (Odds Ratio).
* Nếu Odds Ratio = 2: Khi X tăng 1 đơn vị, khả năng xảy ra sự kiện Y tăng gấp đôi.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

# --- GIẢ LẬP DỮ LIỆU TÍN DỤNG ---
# X1: Thu nhập, X2: Điểm tín dụng cũ
# Y: 0 (Trả được nợ), 1 (Vỡ nợ - Default)

n_credit = 500
income = np.random.normal(50, 15, n_credit)
credit_score = np.random.normal(650, 50, n_credit)

# Tạo quy luật: Thu nhập thấp + Điểm tín dụng thấp -> Dễ vỡ nợ (Y=1)
# Hàm Logit
z = -10 + 0.05 * income + 0.01 * credit_score + np.random.normal(0, 2, n_credit)
prob = 1 / (1 + np.exp(z)) # Chuyển sang xác suất vỡ nợ thấp (để code logic ngược lại xíu cho thực tế)
# Thực tế: Thu nhập cao -> Vỡ nợ thấp. Nên hệ số income nên làm cho xác suất vỡ nợ giảm.

# Sửa lại logic sinh dữ liệu cho đúng thực tế:
# Z càng lớn -> Khả năng vỡ nợ càng cao. 
# Thu nhập cao (Hệ số âm), Điểm cao (Hệ số âm) -> Z nhỏ -> Xác suất vỡ nợ thấp
z_real = 5 - 0.08 * income - 0.005 * credit_score + np.random.normal(0, 1, n_credit)
probability_default = 1 / (1 + np.exp(-z_real))

# Chuyển xác suất thành nhãn 0/1 (Ngưỡng 0.5)
default_status = (probability_default > 0.5).astype(int)

df_credit = pd.DataFrame({'Income': income, 'Score': credit_score, 'Default': default_status})

# Chia Train/Test
X_train, X_test, y_train, y_test = train_test_split(df_credit[['Income', 'Score']], df_credit['Default'], test_size=0.3, random_state=42)

# Huấn luyện mô hình
logit_model = LogisticRegression()
logit_model.fit(X_train, y_train)

# Dự báo trên tập Test
y_pred = logit_model.predict(X_test)
y_prob = logit_model.predict_proba(X_test)[:, 1]

print("Hệ số hồi quy (Income, Score):", logit_model.coef_)
print("Intercept:", logit_model.intercept_)

### 3.2.3. Đánh giá Mô hình Phân loại
Khác với hồi quy, chúng ta không dùng $R^2$. Chúng ta dùng:
1.  **Confusion Matrix (Ma trận nhầm lẫn):** Bao nhiêu trường hợp dự báo đúng/sai?
2.  **Accuracy:** Độ chính xác tổng thể.
3.  **ROC & AUC:** Khả năng phân loại của mô hình ở các ngưỡng cắt khác nhau.

In [ ]:
# Vẽ Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Không vỡ nợ', 'Vỡ nợ'], 
            yticklabels=['Không vỡ nợ', 'Vỡ nợ'])
plt.ylabel('Thực tế')
plt.xlabel('Dự báo')
plt.title('Confusion Matrix')
plt.show()

# Vẽ đường cong ROC
fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate (Tỷ lệ báo động giả)')
plt.ylabel('True Positive Rate (Tỷ lệ phát hiện đúng)')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

## 3.3. Phân tích Biệt số (Linear Discriminant Analysis - LDA)

*Phần này giới thiệu khái niệm, chi tiết sẽ đi sâu ở phần giảm chiều dữ liệu.*

**Sự khác biệt với Logistic:**
* **Logistic:** Tập trung vào xác suất $P(Y=1|X)$. Tốt cho bài toán phân loại nhị phân (2 nhóm).
* **LDA:** Tập trung vào tìm trục (không gian) để tách biệt các nhóm rõ ràng nhất. Tốt khi có nhiều hơn 2 nhóm (Ví dụ: Khách hàng VIP, Khách hàng thường, Khách hàng rủi ro).

---

## 3.4. Tổng kết Chương 3

1.  Dùng **Hồi quy tuyến tính** khi muốn dự báo một con số cụ thể (Doanh thu, Giá cả).
2.  Dùng **Hồi quy Logistic** khi muốn dự báo xác suất sự kiện (Có/Không).
3.  Luôn chia dữ liệu thành **Train/Test** để kiểm tra độ chính xác thực tế, tránh hiện tượng học vẹt (Overfitting).

### Bài tập thực hành
* Sử dụng bộ dữ liệu `Housing` trên Kaggle.
* Chạy mô hình Hồi quy đa biến để dự báo giá nhà.
* Thử loại bỏ bớt các biến có P-value > 0.05 và so sánh xem $R^2$ hiệu chỉnh có tốt lên không?